In [3]:
import gurobipy as gp
import pandas as pd
from code_map import data_handling
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os


In [5]:
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./weekly_collections.pkl")

EOFError: Ran out of input